* Цель:
    Научиться использовать модели скользящего среднего (Moving Average) и ARIMA для прогнозирования временного ряда, 
    используя встроенный датасет flights из библиотеки seaborn, а также освоить автоматический подбор параметров модели ARIMA с помощью auto_arima.
* Описание задания:
    Ваша задача — проанализировать данные о количестве авиапассажиров, построить модель скользящего среднего (MA) и модель ARIMA, 
    автоматически подобрать параметры для ARIMA с помощью библиотеки auto_arima, а затем сравнить прогнозы обеих моделей.
* Шаги:
    1. Загрузка данных:
        - Используйте встроенный датасет flights из библиотеки seaborn.
    2. Простое сглаживание (Moving Average):
        - Примените скользящее среднее с окном 12 месяцев (один год) для данных.
        - Постройте график с исходными данными и результатом применения скользящего среднего.
    3. Подбор параметров ARIMA с помощью auto_arima:
        - Используйте библиотеку pmdarima для автоматического подбора параметров модели ARIMA.
        - Подберите оптимальные параметры ARIMA и выведите их на экран.
    4. Построение и прогноз модели ARIMA:
        - Постройте модель ARIMA с подобранными параметрами.
        - Сделайте прогноз на следующие 12 месяцев и выведите его на график.
    5. Сравнение:
        - Сравните результаты скользящего среднего и ARIMA на одном графике.
        - Сделайте выводы: какие прогнозы более точные и стабильные?